In [1]:
# SETUP
from IPython.display import display
import pandas as pd
import os
from copy import deepcopy
import numpy as np
from math import isclose

os.chdir("..")
#from adaptive_multigrid import give_data_individual_names, find_best_individual_name, build_individual_dict_from_param_grid, update_param_grid
from psuu import scenario_configs, build_next_param_config_code, THRESHOLD_INEQUALITIES_MAP, compute_threshold_inequalities, load_kpis, load_sweep
from model.config.params import *

/Users/seanmcowen/opt/anaconda3/envs/BlockScience/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sweep = "gateway_viability_sweep_ag1_"
(sweep_family,
kpis,
param_config,
next_name,
variable_params,
control_params,
threshold_inequalities,
threshold_parameters) = load_sweep(sweep)

df_thresholds = compute_threshold_inequalities(
    kpis, variable_params, threshold_parameters, threshold_inequalities
)
display(df_thresholds)
print(df_thresholds.mean())

servicer_npv_success  \
param_session_token_bucket_coefficient param_gateway_fee_per_relay param_application_fee_per_relay param_gateway_minimum_stake param_minimum_application_stake                         
25                                     10                          10                              1.000000e+11                1.000000e+10                                     True   
                                                                                                                               2.000000e+10                                     True   
                                                                                                   2.000000e+11                1.000000e+10                                     True   
                                                                                                                               2.000000e+10                                     True   
                                                                   100                             1.000000e+11                1.000000e+10                                     True   
                                                                                                                               2.000000e+10                                     True   
                                                                                                   2.000000e+11                1.000000e+10                                     True   
                                                                                                                               2.000000e+10                                     True   
                                       100                         10                              1.000000e+11                1.000000e+10                                     True   
                                                                                                                               2.000000e+10                                     True   
                                                                                                   2.000000e+11                1.000000e+10                                     True   
                                                                                                                               2.000000e+10                                     True   
                                                                   100                             1.000000e+11                1.000000e+10                                     True   
                                                                                                                               2.000000e+10                                     True   
                                                                                                   2.000000e+11                1.000000e+10                                     True   
                                                                                                                               2.000000e+10                                     True   
400                                    10                          10                              1.000000e+11                1.000000e+10                                     True   
                                                                                                                               2.000000e+10                                     True   
                                                                                                   2.000000e+11                1.000000e+10                                     True   
                                                                                                                               2.000000e+10                                     True   
                                                                   100                             1.000000e+11 

servicer_npv_success              1.000
servicer_capital_costs_success    0.625
gateway_npv_success               0.000
dtype: float64


In [3]:
sweep = "network_failures_service_ag1_"
(sweep_family,
kpis,
param_config,
next_name,
variable_params,
control_params,
threshold_inequalities,
threshold_parameters) = load_sweep(sweep)

df_thresholds = compute_threshold_inequalities(
    kpis, variable_params, threshold_parameters, threshold_inequalities
)
display(df_thresholds)
print(df_thresholds.mean())

servicer_slashing_cost_success  \
param_slash_fraction_downtime param_downtime_jail_duration param_max_chains_servicer                                   
1.000000e-10                  6.000000e+10                 1                                                    True   
                                                           20                                                   True   
                              2.880000e+13                 1                                                    True   
                                                           20                                                   True   
1.000000e-01                  6.000000e+10                 1                                                    True   
                                                           20                                                   True   
                              2.880000e+13                 1                                                    True   
                                                           20                                                   True   

                                                                                      servicer_jailing_cost_success  \
param_slash_fraction_downtime param_downtime_jail_duration param_max_chains_servicer                                  
1.000000e-10                  6.000000e+10                 1                                                   True   
                                                           20                                                  True   
                              2.880000e+13                 1                                                   True   
                                                           20                                                  True   
1.000000e-01                  6.000000e+10                 1                                                   True   
                                                           20                                                  True   
                              2.880000e+13                 1                                                   True   
                                                           20                                                  True   

                                                                                      network_load_balancing_success  
param_slash_fraction_downtime param_downtime_jail_duration param_max_chains_servicer                                  
1.000000e-10                  6.000000e+10                 1                                                   False  
                                                           20                                                  False  
                              2.880000e+13                 1                                                   False  
                                                           20                                                  False  
1.000000e-01                  6.000000e+10                 1                                                   False  
                                                           20                                                  False  
                              2.880000e+13                 1                                                   False  
                                                           20                                                  False

servicer_slashing_cost_success    1.0
servicer_jailing_cost_success     1.0
network_load_balancing_success    0.0
dtype: float64


In [ ]:
sweep = "network_failures_oracle_ag1_"
(sweep_family,
kpis,
param_config,
next_name,
variable_params,
control_params,
threshold_inequalities,
threshold_parameters) = load_sweep(sweep)

df_thresholds = compute_threshold_inequalities(
    kpis, variable_params, threshold_parameters, threshold_inequalities
)
display(df_thresholds)
print(df_thresholds.mean())

In [ ]:
sweep = "network_viability_ag1_"
(sweep_family,
kpis,
param_config,
next_name,
variable_params,
control_params,
threshold_inequalities,
threshold_parameters) = load_sweep(sweep)

df_thresholds = compute_threshold_inequalities(
    kpis, variable_params, threshold_parameters, threshold_inequalities
)
display(df_thresholds)
print(df_thresholds.mean())

In [ ]:
sweep = "servicer_viability_ag1_"
(sweep_family,
kpis,
param_config,
next_name,
variable_params,
control_params,
threshold_inequalities,
threshold_parameters) = load_sweep(sweep)

df_thresholds = compute_threshold_inequalities(
    kpis, variable_params, threshold_parameters, threshold_inequalities
)
display(df_thresholds)
print(df_thresholds.mean())